In [1]:
import numpy as np
import cv2 as cv
import glob
import time
from pathlib import Path
import depthai as dai

In [2]:
'''
    function to capture 10 images from mentioned source 
    - source can be RIGHT or LEFT monochrome camera of OAK D LITE
    - images captured at 1000 sec interval
    
    params : 
    
        src = {'right' || 'left'} (default : right)
        delay = {delay in ms} (default : 1000)
'''

def captureImages(src='right', delay=1000):
    if src != 'right' and src != 'left': 
        print("ENTER CORRECT PARAMS!")
        print("accepted params: {left, right} ")
        print(f"entered src:{src}")
        return;
    
    # Start defining a pipeline
    pipeline = dai.Pipeline()

    # Define a source - mono (grayscale) camera
    # LEFT or RIGHT    
    
    cam = pipeline.createMonoCamera()
    
    if src == 'right' :
        cam.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    else :
        cam.setBoardSocket(dai.CameraBoardSocket.LEFT)

    cam.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)

    # Create output
    xout = pipeline.createXLinkOut()
    xout.setStreamName(src)
    cam.out.link(xout.input)

    # Connect and start the pipeline
    with dai.Device(pipeline,usb2Mode=True) as device:

        # Output queue will be used to get the grayscale frames from the output defined above
        q = device.getOutputQueue(name=src, maxSize=4, blocking=False)

        # Make sure the destination path is present before starting to store the examples
        Path(f"images/{src}").mkdir(parents=True, exist_ok=True)

        for i in range(10):
            # Blocking call, will wait until a new data has arrived
            inSrc = q.get()  
            # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
            frame = inSrc.getCvFrame()
            # Frame is transformed and ready to be shown
            cv.imshow(src, frame)

            cv.imwrite(f"images/{src}/{int(time.time() * 10000)}.png", frame)
            cv.waitKey(delay)  

            cv.destroyAllWindows()            


def captureColorImages(delay=1000):
    
    # Start defining a pipeline
    pipeline = dai.Pipeline()

    # Define a source - color camera
    
    cam = pipeline.createColorCamera()
    cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)

    # Create RGB output
    xout = pipeline.createXLinkOut()
    xout.setStreamName("rgb")
    cam.video.link(xout.input)

    # Connect and start the pipeline
    with dai.Device(pipeline,usb2Mode=True) as device:

        # Output queue will be used to get the color frames from the output defined above
        q = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

        # Make sure the destination path is present before starting to store the examples
        Path(f"images/rgb").mkdir(parents=True, exist_ok=True)

        for i in range(10):
            # Blocking call, will wait until a new data has arrived
            inSrc = q.get()  
            # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
            frame = inSrc.getCvFrame()
            # Frame is transformed and ready to be shown
            imS = cv.resize(frame, (960, 540)) # Resize image
            cv.imshow("rgb", imS)   
#             cv.imshow("rgb", frame)

            cv.imwrite(f"images/rgb/{int(time.time() * 10000)}.png", frame)
            cv.waitKey(delay)  

            cv.destroyAllWindows()

In [3]:
'''
    function to find corners, caliberate and store 
    camera matrix and distortion vector from mentioned source 
    - source can be RIGHT or LEFT monochrome camera or COLOR camera 
    of OAK D LITE
    
    params : 
        images = {array of image paths}
        src = source file {'right' || 'left' || 'rgb'} 
'''

def caliberate(images, src):
    if src != 'right' and src != 'left' and src != 'rgb': 
        print("ENTER CORRECT PARAMS!")
        print("accepted params: {left, right, rgb} ")
        print(f"entered src:{src}")
        return;

    
    # termination criteria
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    notFound = []

#     img_size = () # will be using this for caliberation

    for fname in images:
        img = cv.imread(fname)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        
#         img_size = gray.shape[::-1]
    
        cv.imshow('gray', img)        
        cv.waitKey(1000)
        cv.destroyAllWindows()

        # Finding chess board corners
        ret, corners = cv.findChessboardCorners(gray, (9,6), None)

        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners)

            # Draw and display the corners
            cv.drawChessboardCorners(img, (9,6),corners2, ret)
            cv.imshow('img', img)

            # Saving diplayed corners for future references
            cv.imwrite(f"{fname.split('.')[0]}_corners.png", img)
            cv.waitKey(1000)
            cv.destroyAllWindows()
        else :
            # if corners not found, storing it into a list

            notFound.append(fname)
            print(f"corners not found for {fname}")

    cv.destroyAllWindows()

    # removing the pictures whose corners werent found
    # from the main image list
    for i in notFound:
        images.remove(i)

    # calibration
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        
    for fname in images:
        img = cv.imread(fname)
        h,  w = img.shape[:2]
        newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

        # undistort
        dst = cv.undistort(img, mtx, dist, None, newcameramtx)

        # crop the image
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]
        
        # save new image into file
        cv.imwrite(f"{fname.split('.')[0]}_result.png", dst)
        
        
    # we will be storing the camera matrix and 
    # distortion coefficients for future uses

    print("Saving camera matrix...")
    camera_matrix = np.matrix(mtx)
    with open(f"images/{src}/camera_matrix.txt",'wb') as f:
        for line in camera_matrix:
            np.savetxt(f, line, fmt='%.5f')
            
    print("Saving distortion vector...")
    distortion_vector = np.matrix(dist)
    with open(f"images/{src}/distortion_matrix.txt",'wb') as f:
        for line in distortion_vector:
            np.savetxt(f, line, fmt='%.5f')
            
    
    print("Saving rotational vectors ...")
    rotation_vectors = np.array(rvecs)
    with open(f"images/{src}/rotat_vector.txt",'wb') as f:
        for vector in rotation_vectors:
            vector = np.reshape(vector, (1,3))
            np.savetxt(f, vector, fmt='%.5f')
            
    print("Saving translation vectors...")
    translation_vectors = np.array(tvecs)
    with open(f"images/{src}/trans_vector.txt",'wb') as f:
        for vector in translation_vectors:
            vector = np.reshape(vector, (1,3))
            np.savetxt(f, vector, fmt='%.5f')
            
    print('Done!!')


In [4]:
'''
    calling function to capture images
    
    i used the previously declared function to 
    capture 10 images of a chessboard 
    
    these images are then used to caliberate the camera
    i used a 8x6 chess board for caliberations purposes
    
'''

# capturing images using right monochrome camera
captureImages('right')

# capturing images using left monochrome camera
captureImages('left')

# capturing images using color camera
captureColorImages()


In [5]:
right_images = glob.glob('images/right/*.png')
color_images = glob.glob('images/rgb/*.png')
left_images = glob.glob('images/left/*.png')

# bruh_images = glob.glob('images/bruh/*.png')

caliberate(right_images, 'right')
caliberate(color_images, 'rgb')
caliberate(left_images, 'left')

Saving camera matrix...
Saving distortion vector...
Saving rotational vectors ...
Saving translation vectors...
Done!!
Saving camera matrix...
Saving distortion vector...
Saving rotational vectors ...
Saving translation vectors...
Done!!
Saving camera matrix...
Saving distortion vector...
Saving rotational vectors ...
Saving translation vectors...
Done!!
